<h1>
Capstone Project Final Notebook - Restaurant Clustering  in Toronto
</h1>

<h2>
Install, initialize and import libraries
</h2>

In [8]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Folium installed
Libraries imported.


<h2>
Prepare to use Foursquare APIs
</h2>

<h3>
Set Foursquare Credentials and raise the limit of Foursquare APIs response
</h3>

In [9]:
CLIENT_ID = '0DGJFFPCOKKJS1UZ4GNDEFSTRIPBIW5ZLGWR2DCU303FVXNB' # your Foursquare ID' 
 # your Foursquare ID
CLIENT_SECRET = 'HPABOR4UB2TEO2E12WDHUYL4QPU4N2UAOHRMKXJ421FCKMCI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

<h3>
Create a generic function calling Foursquare API and returning the response as a Pandas DataFrame
</h3>

In [10]:
def get_Venues(url):
    results = requests.get(url).json()
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    dataframe.head()
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    return dataframe_filtered

<h3>
Create a generic function calling previous function and looking for a specific cathegory
</h3>

In [11]:
def get_Venue_byCategory_type(category):
    return get_Venues('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&limit={}&near={},&v={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, LIMIT,near, VERSION, category ))

<h3>
Create a generic function calling previous function and looking for keyword in venue name
</h3>

In [12]:
def get_Venue_byKeyword(keyword):
    return get_Venues('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={},&v={}&query={}'.format(CLIENT_ID, CLIENT_SECRET, near, VERSION, keyword ))

<h2>
Data Collection
</h2>

Looking for Italian restaurants

In [13]:
near = "Toronto, ON, CA"
df_italian = get_Venue_byCategory_type("4bf58dd8d48988d110941735")
df_italian.shape

(50, 15)

Looking for Fast Food restaurants

In [14]:
df_fastFood = get_Venue_byCategory_type("4bf58dd8d48988d16e941735")
df_fastFood.shape

(50, 14)

Looking for Pizza restaurants

In [15]:
df_pizza = get_Venue_byCategory_type("4bf58dd8d48988d1ca941735")
df_pizza.shape

(50, 15)

Looking for Burgers restaurants

In [18]:
df_burger = get_Venue_byCategory_type("4bf58dd8d48988d16c941735")
df_burger.shape

(50, 15)

<h2>
Data Preparation
</h2>

Merging the collected DataFrames

In [19]:
df_locations = df_italian.append(df_fastFood).append(df_pizza).append(df_burger)
df_locations.shape
df_locations.head()

,categories,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name
0,"[{'id': '4bf58dd8d48988d110941735', 'primary':...",4ad4c061f964a520a2f720e3,2373 Yonge St.,CA,Toronto,Canada,NaN,"[2373 Yonge St., Toronto ON M4P 2C8, Canada]","[{'lat': 43.70932882869532, 'lng': -79.3988226...",43.709329,-79.398823,NaN,M4P 2C8,ON,Grazie Ristorante
1,"[{'id': '4bf58dd8d48988d110941735', 'primary':...",5a4fe20a286fda0166f6b594,972 College St,CA,Toronto,Canada,NaN,"[972 College St, Toronto ON M6H 1A5, Canada]","[{'lat': 43.653481, 'lng': -79.42769, 'label':...",43.653481,-79.427690,NaN,M6H 1A5,ON,Giulietta
2,"[{'id': '4bf58dd8d48988d110941735', 'primary':...",5a492661c4df1d5eb27d8324,NaN,CA,Etobicoke,Canada,NaN,"[Etobicoke ON M9C 5H5, Canada]","[{'lat': 43.61422615805952, 'lng': -79.5599425...",43.614226,-79.559943,NaN,M9C 5H5,ON,Scaddabush
3,"[{'id': '4bf58dd8d48988d110941735', 'primary':...",54139d97498e2ddcc2ecceb9,9001 Dufferin Street,CA,Vaughan,Canada,NaN,"[9001 Dufferin Street, Vaughan ON L4J 8X2, Can...","[{'lat': 43.83631383213661, 'lng': -79.4809718...",43.836314,-79.480972,NaN,L4J 8X2,ON,Levetto
4,"[{'id': '4bf58dd8d48988d110941735', 'primary':...",5963b3e52be425495a1ed8a1,1281 Bloor St. W.,CA,Toronto,Canada,Lansdowne,"[1281 Bloor St. W. (Lansdowne), Toronto ON M6H...","[{'lat': 43.65841172348161, 'lng': -79.4424430...",43.658412,-79.442443,Brockton Village,M6H 3Y2,ON,Sugo


Removing duplicate entries

In [20]:
df_uniqueLocations = df_locations.drop_duplicates(subset="name")

In [21]:
df_uniqueLocations.shape

(96, 15)

Keeping only Toronto city venues

In [22]:
df_final = df_uniqueLocations.loc[df_uniqueLocations["location.city"] == "Toronto"]
df_final.shape

(68, 15)

Having a first look at the locations

In [23]:
# define the world map centered around Toronto, Canada with a higher zoom level
# Toronto https://github.com/jasonicarter/toronto-geojson/blob/master/toronto_crs84.geojson
tr_map = folium.Map(location=[43.653908,-79.384293], zoom_start=11)
tr_feature_group = folium.map.FeatureGroup()

# Add post code locations on the map
for i in df_final["id"]:
    # folium_location=[cp.loc[cp["Postcode"] == i, cp["Postcode"]["Latitude"]],cp.loc[cp["Postcode"] == i, ["Longitude"]]]
    lat = df_final.loc[df_final["id"] == i, ["location.lat"]].get_values()
    lon = df_final.loc[df_final["id"] == i, ["location.lng"]].get_values() 
    folium_location=[lat[0][0],lon[0][0]]
    # Add one marker per postal code location
    tr_feature_group.add_child(folium.Marker(location=folium_location,popup=i))

tr_feature_group.add_to(tr_map)

tr_map

<h2>
Running ML algorithm: Density based clustering
</h2>

In [24]:
from sklearn.cluster import DBSCAN 
from sklearn.preprocessing import StandardScaler 

Extract clustered features and prepare them for algorithm training

In [25]:
Coordonates = df_final[["location.lat","location.lng"]]
Coordonates = np.nan_to_num(Coordonates)
Coordonates = StandardScaler().fit_transform(Coordonates)

Length in meters of 1° of latitude = always 111.32 km

Length in meters of 1° of longitude = 40075 km * cos( latitude ) / 360
At Tontoto latitude this makes 103.28330778074714 km per 1°

<h3>
In order to choose the most appropriate clustered areas, a grid based approach is used with different value of eps and minimum number of cluster members.
</h3>

In [27]:
eps_list = [0.1, 0.15,0.2,0.25,0.3,0.35,0.4,0.45 ]
sample_list= [3,4,5]
for e in eps_list:
    for s in sample_list:
        test_db = DBSCAN(eps=e, min_samples=s).fit(Coordonates)
        test_labels = test_db.labels_
        realClusterNum=len(set(test_labels)) - (1 if -1 in test_labels else 0)
        clusterNum = len(set(test_labels)) 
        unique, counts = np.unique(test_labels, return_counts=True)
        label_dict = dict(zip(unique, counts))
        print("For eps "+str(e)+" and minimum samples "+str(s)+" ,the number of clusters is "+str(clusterNum)+" and outliers are "+str(label_dict[-1]) )


For eps 0.1 and minimum samples 3 ,the number of clusters is 5 and outliers are 43
For eps 0.1 and minimum samples 4 ,the number of clusters is 4 and outliers are 49
For eps 0.1 and minimum samples 5 ,the number of clusters is 4 and outliers are 49
For eps 0.15 and minimum samples 3 ,the number of clusters is 5 and outliers are 34
For eps 0.15 and minimum samples 4 ,the number of clusters is 3 and outliers are 40
For eps 0.15 and minimum samples 5 ,the number of clusters is 3 and outliers are 42
For eps 0.2 and minimum samples 3 ,the number of clusters is 3 and outliers are 30
For eps 0.2 and minimum samples 4 ,the number of clusters is 4 and outliers are 31
For eps 0.2 and minimum samples 5 ,the number of clusters is 3 and outliers are 34
For eps 0.25 and minimum samples 3 ,the number of clusters is 2 and outliers are 24
For eps 0.25 and minimum samples 4 ,the number of clusters is 3 and outliers are 24
For eps 0.25 and minimum samples 5 ,the number of clusters is 3 and outliers are 2

Final pass with selected hyper parametes

In [45]:
db = DBSCAN(eps=0.2, min_samples=4).fit(Coordonates)

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

Reinjecting cluster numbers in prepared DataFrame

In [46]:
df_final = df_final.assign(Cluster=labels)

<h3>
Map with  clusters andthe outliers painted in black.
</h3>

In [47]:
# define the world map centered around Toronto, Canada with a higher zoom level
# Toronto https://github.com/jasonicarter/toronto-geojson/blob/master/toronto_crs84.geojson
tr_cmap = folium.Map(location=[43.653908,-79.384293], zoom_start=12)
#df_uniqueLocations.set_index("id")
tr_feature_cgroup = folium.map.FeatureGroup()
cluster_colors =  ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
             'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
             'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
             'gray', 'lightgray']

# Add post code locations on the map
for i in df_final["id"]:
    lat = df_final.loc[df_final["id"] == i, ["location.lat"]].get_values()
    lon = df_final.loc[df_final["id"] == i, ["location.lng"]].get_values() 
    clust = df_final.loc[df_final["id"] == i, ["Cluster"]].get_values() 
    rname = df_final.loc[df_final["id"] == i, ["name"]].get_values().item()
    folium_location=[lat[0][0],lon[0][0]]
    popup = folium.Popup(rname, parse_html=True)
    # Add one marker per postal code location
    if clust == -1:
        tr_feature_cgroup.add_child(folium.Marker(location=folium_location,popup=popup,icon=folium.Icon(color='black')))
    else:
        tr_feature_cgroup.add_child(folium.Marker(location=folium_location,popup=popup,icon=folium.Icon(color=cluster_colors[clust.item()])))

tr_feature_cgroup.add_to(tr_cmap)

tr_cmap

Determining clusters in and out zones in selected cluster Blue (cluster #0)  and Green (cluster #1)

In [90]:
Cluster2 = df_final.loc[df_final["Cluster"] == 2]
Cluster1 = df_final.loc[df_final["Cluster"] == 1]

In [91]:
cluster2_western = Cluster2.loc[[Cluster2["location.lng"].idxmin()]]
cluster2_eastern = Cluster2.loc[[Cluster2["location.lng"].idxmax()]]
cluster1_western = Cluster1.loc[[Cluster1["location.lng"].idxmin()]]
cluster1_eastern = Cluster1.loc[[Cluster1["location.lng"].idxmax()]]

In [92]:
west_pos2 = [cluster2_western["location.lat"].values[0],cluster2_western["location.lng"].values[0]]
east_pos2 = [cluster2_eastern["location.lat"].values[0],cluster2_eastern["location.lng"].values[0]]
west_pos1 = [cluster1_western["location.lat"].values[0],cluster1_western["location.lng"].values[0]]
east_pos1 = [cluster1_eastern["location.lat"].values[0],cluster1_eastern["location.lng"].values[0]]

In [93]:
toronto_aiport = [43.6777176,-79.6248197]
kew_garden_park = [43.668725, -79.298501]

Final Map after removing unselected cluster and outliers.

In [97]:
# define the world map centered around Toronto, Canada with a higher zoom level
# Toronto https://github.com/jasonicarter/toronto-geojson/blob/master/toronto_crs84.geojson
tr_cmap = folium.Map(location=[43.653908,-79.384293], zoom_start=12)
#df_uniqueLocations.set_index("id")
tr_feature_cgroup = folium.map.FeatureGroup()
cluster_colors =  ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
             'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
             'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
             'gray', 'lightgray']

# Add post code locations on the map
for i in df_final["id"]:
    lat = df_final.loc[df_final["id"] == i, ["location.lat"]].get_values()
    lon = df_final.loc[df_final["id"] == i, ["location.lng"]].get_values() 
    clust = df_final.loc[df_final["id"] == i, ["Cluster"]].get_values() 
    rname = df_final.loc[df_final["id"] == i, ["name"]].get_values().item()
    folium_location=[lat[0][0],lon[0][0]]
    popup = folium.Popup(rname, parse_html=True)
    # Add one marker per postal code location
    if clust == 1 or clust==2 :
        tr_feature_cgroup.add_child(folium.Marker(location=folium_location,popup=popup,icon=folium.Icon(color=cluster_colors[clust.item()])))
tr_feature_cgroup.add_child(folium.Marker(location=kew_garden_park,popup="Kew Gardens Park"))
        
tr_feature_cgroup.add_to(tr_cmap)

folium.PolyLine(locations=[toronto_aiport, west_pos1], color='blue').add_to(tr_cmap)
folium.PolyLine(locations=[east_pos1, west_pos2], color='blue').add_to(tr_cmap)
folium.PolyLine(locations=[east_pos2, kew_garden_park], color='blue').add_to(tr_cmap)

tr_cmap